In [2]:
#import api_files.ml_library as calc
import pandas as pd
from notebooks.train_test_divide import extract_spammers_scrapers, extract_normal, split_train_test
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os

path = os.path.dirname(os.path.abspath("__file__"))
dir = path.replace('notebooks', 'api_files')
os.chdir(dir)
from api_files.ml_library import trim, read_csv_file, clean_reqlogs, extract_users, split_user_df, calc_avg_timediff, avg_tokens_5mins, longest_consec, get_variance_score, sequence_time_length

In [6]:
"""Code for fetching the data"""
#Fetch data from csv file
data = trim(read_csv_file('csv_files/requests.csv'))
#Extracts a list of all unique users in dataframe
users = extract_users(data)

#Create a datachunk for testing the API
df_chunks = split_user_df(data, users[1], 28) 
df_chunks[1].to_csv('csv_files/datachunk.csv', index=False)

vect = HashingVectorizer(norm=None, alternate_sign=False)

In [ ]:
#32893 requests made by user-1003
data.loc[data['userID'] == 'user-1003']

#32714 requests made by user-1004
data.loc[data['userID'] == 'user-1004']

#68413 requests made by user-1005
data.loc[data['userID'] == 'user-1005']

In [6]:
"""
Code for creating csv-file containing information about average request frequency, average number of tokens per 5 mins, 
longest consecutive chain of repetitive requests for every chunk of 50 requests for each user
"""

df_calculations = pd.DataFrame()
for user in users:
    data_chunks = split_user_df(data, user, 30)
    avg_req_frequences = []
    avg_tokens = []
    repetitions = []
    var_scores = []
    sequence_time = []

    for i in range(0, len(data_chunks)):
        avg_req_frequences.append(calc_avg_timediff(data_chunks[i]))
        avg_tokens.append(avg_tokens_5mins(data_chunks[i]))
        repetitions.append(longest_consec(data_chunks[i]))
        var_scores.append(get_variance_score(data_chunks[i], vect))
        sequence_time.append(sequence_time_length(data_chunks[i]))
    
    user_df = pd.DataFrame()
    user_df['request_freq'] = avg_req_frequences
    user_df['avg_tokens'] = avg_tokens
    user_df['longest_consec'] = repetitions
    user_df['var_score'] = var_scores
    user_df['sequence_time'] = sequence_time
    user_df['user'] = user
    df_calculations = pd.concat([df_calculations, user_df])

df_calculations.reset_index(inplace=True)
df_calculations = df_calculations.drop(columns='index')
df_calculations.to_csv('csv_files/calculations80.csv', index=False) 

In [ ]:
data_calc = pd.read_csv('csv_files/calculations20.csv')
data_calc

In [8]:
df_spam = extract_spammers_scrapers(data_calc)
split_train_test(extract_normal(data_calc, df_spam), df_spam)